### Information regarding the code
This script gathers the videos published by selected creators for the study. 
As the initial JSONL file is heavy (~97GB), I process to cut the data retrieving process into multiple iterations. Each iteration is saved into a csv file.

In [1]:
import pandas as pd
import gzip
import json
import csv

In [2]:
#Channel list
db_sample=pd.read_csv('channels_criteria.csv', sep=';')
db_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95859 entries, 0 to 95858
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          95859 non-null  int64  
 1   category_cc         95738 non-null  object 
 2   join_date           95859 non-null  object 
 3   channel             95859 non-null  object 
 4   name_cc             95853 non-null  object 
 5   subscribers_cc      95859 non-null  int64  
 6   videos_cc           95859 non-null  int64  
 7   subscriber_rank_sb  95859 non-null  float64
 8   weights             95859 non-null  float64
 9   prior_crea_date     95859 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 7.3+ MB


In [3]:
list_channel=db_sample['channel'].tolist()
len(list_channel)

95859

In [4]:
# Chunk channels list - returns a list of 12 levels of the initial list
chunks = [list_channel[x:x+10000] for x in range(0, len(list_channel), 10000)]


In [5]:
# As each line in the jsonl file is a dict: need to create a list for each key
df_cat=[]
df_chan=[]
df_crawl=[]
df_desc=[]
df_dislike=[]
df_id=[]
df_duration=[]
df_like=[]
df_tags=[]
df_title=[]
df_upload=[]
df_views=[]

In [6]:
#Decompressing the jsonl.gz file

def is_a_needed_json(json_item):
    return json_item['channel_id'] in chunks[9]
# ^ If false it returns none (Lighter conditionnal loop)
# Will need to change the index number after each run


with gzip.open('yt_metadata_en.jsonl.gz', 'rb') as data:
    for line in data:
        video=json.loads(line)
        #video is a dict type - print(video.keys())
        
        if is_a_needed_json(video):
            df_cat.append(video['categories'])
            df_chan.append(video['channel_id'])
            df_crawl.append(video['crawl_date'])
            df_desc.append(video['description'])
            df_dislike.append(video['dislike_count'])
            df_id.append(video['display_id'])
            df_duration.append(video['duration'])
            df_like.append(video['like_count'])
            df_tags.append(video['tags'])
            df_title.append(video['title'])
            df_upload.append(video['upload_date'])
            df_views.append(video['view_count'])
        else:
            pass

In [7]:
db_video=pd.DataFrame(list(zip(df_cat, df_chan,df_crawl, df_desc, df_dislike, df_id, df_duration, df_like, df_tags, df_title, df_upload, df_views)),
                     columns =['category', 'channel', 'date_crawled','description', 'dislikes', 'id', 'duration', 'like', 'tags', 'title', 'upload', 'view'])

db_video.head()
db_video.to_csv('video_dataset_chunk_9.csv', sep=';')

In [8]:
db_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2373548 entries, 0 to 2373547
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   category      object 
 1   channel       object 
 2   date_crawled  object 
 3   description   object 
 4   dislikes      float64
 5   id            object 
 6   duration      int64  
 7   like          float64
 8   tags          object 
 9   title         object 
 10  upload        object 
 11  view          float64
dtypes: float64(3), int64(1), object(8)
memory usage: 217.3+ MB
